In [96]:
import pandas as pd
from datetime import datetime
from slugify import slugify
from currency_converter import CurrencyConverter
import numpy as np

In [97]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [98]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [99]:
df = pd.read_csv('csv/products.csv')
images = pd.read_csv('out/productsImages.csv')
price = pd.read_csv('out/price.csv')
place = pd.read_csv('out/place.csv')

In [100]:
df['Price 1'] = df['Price 1'].str.replace(",", ".").astype(float)
df['Price 2'] = df['Price 2'].str.replace(",", ".").astype(float)
df['Price 3'] = df['Price 3'].str.replace(",", ".").astype(float)
df['Price 4'] = df['Price 4'].str.replace(",", ".").astype(float)
df['Price 5'] = df['Price 5'].str.replace(",", ".").astype(float)

In [101]:
df['Rating 1'] = df['Rating 1'].astype(float)
df['Rating 2'] = df['Rating 2'].astype(float)
df['Rating 3'] = df['Rating 3'].astype(float)
df['Rating 4'] = df['Rating 4'].astype(float)
df['Rating 5'] = df['Rating 5'].astype(float)

In [102]:
c = CurrencyConverter()
def convert_currency(x, y):
    try:
        return c.convert(float(x), str(y), 'USD')
    except:
        return np.nan

In [103]:
df['Price 1 USD'] = df.apply(lambda x: convert_currency(x['Price 1'], x['Curr1']), axis=1).astype(float).round(2)
df['Price 2 USD'] = df.apply(lambda x: convert_currency(x['Price 2'], x['Curr2']), axis=1).astype(float).round(2)
df['Price 3 USD'] = df.apply(lambda x: convert_currency(x['Price 3'], x['Curr3']), axis=1).astype(float).round(2)
df['Price 4 USD'] = df.apply(lambda x: convert_currency(x['Price 4'], x['Curr4']), axis=1).astype(float).round(2)
df['Price 5 USD'] = df.apply(lambda x: convert_currency(x['Price 5'], x['Curr5']), axis=1).astype(float).round(2)

In [104]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()

In [105]:
df1['minPrice'] = df[['Price 1 USD', 'Price 2 USD', 'Price 3 USD', 'Price 4 USD', 'Price 5 USD']]\
[(df['Price 1 USD'].notnull()) | \
 (df['Price 2 USD'].notnull()) | \
 (df['Price 3 USD'].notnull()) | \
 (df['Price 4 USD'].notnull()) | \
 (df['Price 5 USD'].notnull())
].min(axis=1).astype(float).round(2)
df1['id'] = df['ID'].astype(str) + '_' + df1['minPrice'].astype(str)

In [106]:
df1.shape

(1532, 2)

In [107]:
df2['maxPrice'] = df[['Price 1 USD', 'Price 2 USD', 'Price 3 USD', 'Price 4 USD', 'Price 5 USD']]\
[(df['Price 1 USD'].notnull()) | \
 (df['Price 2 USD'].notnull()) | \
 (df['Price 3 USD'].notnull()) | \
 (df['Price 4 USD'].notnull()) | \
 (df['Price 5 USD'].notnull())
].max(axis=1).astype(float).round(2)
df2['id'] = df['ID'].astype(str) + '_' + df2['maxPrice'].astype(str)

In [108]:
df2.shape

(1532, 2)

In [109]:
df3['avgPrice'] = df[['Price 1 USD', 'Price 2 USD', 'Price 3 USD', 'Price 4 USD', 'Price 5 USD']]\
[(df['Price 1 USD'].notnull()) | \
 (df['Price 2 USD'].notnull()) | \
 (df['Price 3 USD'].notnull()) | \
 (df['Price 4 USD'].notnull()) | \
 (df['Price 5 USD'].notnull())
].mean(axis=1).astype(float).round(2)
df3['id'] = df['ID'].astype(str) + '_' + df3['avgPrice'].astype(str)

In [110]:
df3.shape

(1532, 2)

In [111]:
df1.rename(columns = {'minPrice':'amount'}, inplace = True)
df2.rename(columns = {'maxPrice':'amount'}, inplace = True)
df3.rename(columns = {'avgPrice':'amount'}, inplace = True)

In [112]:
newDf1 = pd.concat([df1, df2, df3])

In [113]:
newDf1['ID'] = newDf1['id']
newDf1['id'] = [i for i in range(price.shape[0]+1, price.shape[0]+1 + newDf1.shape[0])]
newDf1.rename(columns = {'Price':'amount'}, inplace = True)
newDf1['currencyId'] = 'USD'
newDf1['amountMainCurrency'] = newDf1['amount']
newDf1['mainCurrencyId'] = 'USD'

In [114]:
df['avgRating'] = df[['Rating 1', 'Rating 2', 'Rating 3', 'Rating 4', 'Rating 5']]\
[(df['Rating 1'].notnull()) | \
 (df['Rating 2'].notnull()) | \
 (df['Rating 3'].notnull()) | \
 (df['Rating 4'].notnull()) | \
 (df['Rating 5'].notnull())
].mean(axis=1)

In [115]:
df['reviewCount'] = df[['Number Review 1', 'Number Review 2', 'Number Review 3', 'Number Review 4', 'Number Review 5']]\
[(df['Number Review 1'].notnull()) | \
 (df['Number Review 2'].notnull()) | \
 (df['Number Review 3'].notnull()) | \
 (df['Number Review 4'].notnull()) | \
 (df['Number Review 5'].notnull())
].count(axis=1).astype(int)

In [116]:
newDf = df[['Name', 'E-Tickets', 'Description', 'ID', 'avgRating', 'reviewCount']]

In [117]:
columns = ['createdDate', 'updatedDate', 'internalNote', 'creatorId', 'statusId', \
          'slug', 'shortDescription', 'alternativeName', 'mainImageId', 'placeId']

In [118]:
newDf = pd.concat([newDf, pd.DataFrame(columns = columns)])

In [119]:
newDf.rename(columns = {'Name':'name', 'E-Tickets' : 'onlineTicketUrl', \
                    'Description' : 'description', 'ID' : 'id', 'avgRating' : 'averageRating'}, inplace = True)

In [120]:
import random
import time
 
 
def str_time_prop(start, end, time_format, prop):
    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(time_format, time.localtime(ptime))
 
 
def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)
 

In [121]:
newDf['createdDate'] = [random_date("2021-1-1", "2022-4-30", random.random()) for i in range(1, newDf.shape[0] + 1)]
newDf['updatedDate'] = [i for i in newDf['createdDate']]
newDf['internalNote'] = ''
newDf['statusId'] = 4
newDf['creatorId'] = 900
newDf['id'] = newDf.id.astype(int)
newDf['storyCount'] = newDf['reviewCount']

In [122]:
newDf['storyCount'].fillna("0", inplace = True)
newDf['reviewCount'].fillna("0", inplace = True)

In [123]:
def get_place_id(value):
    try:
        return place[:][place['CoordId'] == int(value)]['CoordId'].values[0]
    except:
        return ''

newDf['placeId'] = newDf['id'].apply(get_place_id)

In [124]:
def set_slug_id():
    for index, row in newDf.iterrows():
        newDf.loc[index, ['slug']] = [slugify(row['name'])]
        
set_slug_id()

In [125]:
newDf['new_slug'] = newDf.groupby(['slug']).cumcount() + 1 

In [126]:
def remove_slug_duplicates(slug, increment):
    try:
        if int(increment) == 1:
            return slug
        if int(increment) > 1:
            return slug + str(int(increment)-1)
    except:
        pass
    
newDf['slug'] = newDf[['slug','new_slug']].apply(lambda x: remove_slug_duplicates(*x), axis=1)

In [127]:
def get_short_description(value):
    return str(value)[:200]

newDf['shortDescription'] = newDf['description'].apply(get_short_description)

In [128]:
def get_alternative_name(value):
    temp = value.split(' ')
    return '; '.join(str(e) for e in temp)

newDf['alternativeName'] = newDf['name'].apply(get_alternative_name)

In [129]:
def get_main_image_id(value):
    try:
        tempName = 'product_' + str(value) + '_1'
        return images[:][images['name'].str.contains(tempName)]['id'].values[0]
    except:
        return ''
    
newDf['mainImageId'] = newDf['id'].apply(get_main_image_id)

In [130]:
copy = newDf1.drop(['ID'], axis=1)

In [131]:
def get_price_id(value):
    return newDf1[:][newDf1['ID'] == str(value)]['id'].values[0]

In [132]:
import re

In [133]:
def get_first_number(value):
    return re.findall(r'^\D*(\d+)', str(value))[0]

In [134]:
newDf1['temp_id'] = newDf1['ID'].apply(get_first_number)

In [135]:
p1 = newDf1[['temp_id', 'id']][0:1521]
p2 = newDf1[['temp_id', 'id']][1521:3042]
p3 = newDf1[['temp_id', 'id']][3042:4563]

In [136]:
p1.to_csv('out/p1.csv', index=False, encoding='utf-8')
p2.to_csv('out/p2.csv', index=False, encoding='utf-8')
p3.to_csv('out/p3.csv', index=False, encoding='utf-8')

In [137]:
newDf.to_csv('out/products.csv', index=False, encoding='utf-8')
newDf1.to_csv('out/price2.csv', index=False, encoding='utf-8')

In [138]:
newDf[:][newDf['id'] == 181]

,name,onlineTicketUrl,description,id,averageRating,reviewCount,createdDate,updatedDate,internalNote,creatorId,statusId,slug,shortDescription,alternativeName,mainImageId,placeId,storyCount,new_slug
180,Surfing Lesson,NaN,Fulfill your dreams to master surfing at the m...,181,4.8,5.0,2022-04-16,2022-04-16,,900,4,surfing-lesson1,Fulfill your dreams to master surfing at the m...,Surfing; Lesson,,,5.0,2


In [139]:
copy.to_csv('out/price3.csv', index=False, encoding='utf-8')

In [140]:
copy.shape

(4596, 5)

In [141]:
tempDf = newDf[['id', 'mainImageId']][newDf['mainImageId'] != '']

In [143]:
tempDf.to_csv('out/products_i.csv', index=False, encoding='utf-8')

In [144]:
tempDf.head()

,id,mainImageId
0,1,1360
1,2,4472
2,3,1222
3,4,2661
4,5,2000
